In [1]:
import psycopg2, sqlite3, sys, pytz
from datetime import datetime
from uuid import uuid4
import pandas as pd

In [2]:
def is_dst(aware_dt):
    assert aware_dt.tzinfo is not None
    assert aware_dt.tzinfo.utcoffset(aware_dt) is not None
    return bool(aware_dt.dst())

In [3]:
def createTeam1ResultsMethods():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        # Create cluster geometry table
        cur.execute("""CREATE TABLE team1_results_methods (
                resultid SERIAL PRIMARY KEY,
                resultuuid uuid NOT NULL,
                samplingfeatureexternalidentifier VARCHAR (255) NOT NULL,
                samplingfeaturecode VARCHAR (50) NOT NULL,
                samplingfeaturename VARCHAR (255) NOT NULL,
                variablenamecv VARCHAR (255) NOT NULL,
                variablecode VARCHAR (255) NOT NULL,
                cv_variable_term VARCHAR (255) NOT NULL,
                datavalue DOUBLE PRECISION NOT NULL,
                resultdatetime TIMESTAMP WITH TIME ZONE,
                datasetcode VARCHAR (50) NOT NULL,
                sitetypecv VARCHAR (255) NOT NULL,
                propertyvalue VARCHAR (255) NOT NULL,
                methodid INTEGER NOT NULL,
                methodcode VARCHAR (50) NOT NULL,
                network_class_id INTEGER NOT NULL,
                network_subclass_id INTEGER NOT NULL,
                resulttypecv VARCHAR (25) NOT NULL,
                sampledmediumcv VARCHAR (25),
                statuscv VARCHAR (15),
                unitid INTEGER NOT NULL,
                unitabbreviation VARCHAR (25) NOT NULL,
                site_methodid INTEGER NOT NULL,
                site_methodcode VARCHAR (50) NOT NULL
            )""")

        cur.execute("""COMMIT""")
        cur.execute("""ANALYZE team1_results_methods""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()

In [4]:
createTeam1ResultsMethods()

In [5]:
def getTeam1ConcResultsMethods():
    conn = None
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        cur.execute("""SELECT q.epa_srcid, q.station_id, q.station_name, q.analyte_name,
                       q.abbreviation, q.cas, q.conc_ppt, q.sample_date, q.lab, g.src_type, g.depth
                       FROM quant_summary q 
                       INNER JOIN geolocation g ON g.EPA_srcID = q.EPA_srcID
                       WHERE q.conc_ppt NOT NULL
                       ORDER BY q.epa_srcid""")

        rows = cur.fetchall()
        cur.close()
        conn.close()
     
        df = pd.DataFrame(columns = ['samplingfeatureexternalidentifier', 'samplingfeaturecode',
                                     'samplingfeaturename', 'variablenamecv', 'variablecode',
                                     'cv_variable_term', 'datavalue', 'resultdatetime', 'datasetcode',
                                     'sitetypecv', 'propertyvalue', 'methodid', 'methodcode', 
                                     'network_class_id', 'network_subclass_id', 'resulttypecv',
                                     'sampledmediumcv', 'statuscv', 'unitid','unitabbreviation',
                                     'site_methodid', 'site_methodcode'])
                                     #'class', 'well_i_rat', 'intk_loc']) 
        for row in rows:
            variablecode = row[4]
            if variablecode == 'NVHOS':
                cv_variable_term = '801209-99-4'
            else:
                cv_variable_term = row[5]
                
            year,month,day = row[7].split('-')
            naive = datetime(int(year),int(month),int(day))
            eastern = pytz.timezone('America/New_York')
            aware = eastern.localize(naive, is_dst=None)
            resultdatetime = aware.replace(hour=12, minute=0, second=0)
            #is_dst(resultdatetime)
            
            samplingfeaturecode = row[1]+'_Specimen'
            datasetcode = 'T1Data_'+str(row[8])+'_'+samplingfeaturecode
            src_type = row[9]
            
            if src_type == 'G':
                sitetypecv = 'Well'
                methodid = 10004
                methodcode = 'mass-spec'
                site_methodid = 10003
                site_methodcode = 'grab-gw'
                # From network_classes table
                network_class_id = 102
                network_subclass_id = 105
            elif src_type == 'S':
                sitetypecv = 'Stream'
                methodid = 10004
                methodcode = 'mass-spec'
                site_methodid = 10005
                site_methodcode = 'grab-sw'
                # From network_classes table
                network_class_id = 102
                network_subclass_id = 106
            else:
                sitetypecv = 'None'
                methodid = NaN
                methodcode = 'None'
                
            resulttypecv = 'Measurement'
            sampledmediumcv = 'Liquid aqueous'
            statuscv = 'Public'
            unitid = 2605
            unitabbreviation = 'ppt'
            
            df = df.append({'samplingfeatureexternalidentifier':row[0],'samplingfeaturecode':samplingfeaturecode,
                            'samplingfeaturename':row[2],'variablenamecv':row[3],'variablecode':variablecode,
                            'cv_variable_term':cv_variable_term, 'datavalue':row[6],
                            'resultdatetime':resultdatetime,'datasetcode':datasetcode,'sitetypecv':sitetypecv, 
                            'propertyvalue':row[10],'methodid':methodid,'methodcode':methodcode, 
                            'network_class_id':network_class_id,'network_subclass_id':network_subclass_id,
                            'resulttypecv':resulttypecv,'sampledmediumcv':sampledmediumcv, 
                            'statuscv':statuscv,'unitid':unitid,'unitabbreviation':unitabbreviation,
                            'site_methodid':site_methodid,'site_methodcode':site_methodcode},
                            #'class': row[11], 'well_i_rat': row[12], 'intk_loc': row[13]}, 
                           ignore_index = True)
        
        return df
    except sqlite3.Error as e:
        print(e)

In [6]:
df = getTeam1ConcResultsMethods()
df.head()

,samplingfeatureexternalidentifier,samplingfeaturecode,samplingfeaturename,variablenamecv,variablecode,cv_variable_term,datavalue,resultdatetime,datasetcode,sitetypecv,...,methodcode,network_class_id,network_subclass_id,resulttypecv,sampledmediumcv,statuscv,unitid,unitabbreviation,site_methodid,site_methodcode
0,11736,NC0105010_Specimen,"WEST JEFFERSON, TOWN OF",Perfluorobutanesulfonic acid,PFBS,375-73-5,3.399,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0105010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2605,ppt,10003,grab-gw
1,11736,NC0105010_Specimen,"WEST JEFFERSON, TOWN OF",Perfluorohexanesulfonic acid,PFHxS,355-46-4,4.438,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0105010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2605,ppt,10003,grab-gw
2,12111,NC0111020_Specimen,"BLACK MOUNTAIN, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,2.200,2019-06-24 12:00:00-04:00,T1Data_Ferguson_NC0111020_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2605,ppt,10003,grab-gw
3,12111,NC0111020_Specimen,"BLACK MOUNTAIN, TOWN OF",Perfluoroundecanoic acid,PFUnDA,2058-94-8,1.300,2019-06-24 12:00:00-04:00,T1Data_Ferguson_NC0111020_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2605,ppt,10003,grab-gw
4,17170,NC0156025_Specimen,"OLD FORT, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,1.800,2019-06-24 12:00:00-04:00,T1Data_Ferguson_NC0156025_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2605,ppt,10003,grab-gw


In [7]:
outfile = '../ncpfast-db/team1ConcResultsMethods.csv'
df.to_csv(outfile, header=True, index=False)

In [8]:
len(df['datavalue'])

815

In [9]:
def getTeam1RlResultsMethods():
    conn = None
    try:
        conn = sqlite3.connect('../RoundOneDB/RoundOneDB_final.db')
        cur = conn.cursor()
        cur.execute("""SELECT q.epa_srcid, q.station_id, q.station_name, q.analyte_name,
                       q.abbreviation, q.cas, q.rl_ppt, q.sample_date, q.lab, g.src_type, g.depth
                       FROM quant_summary q 
                       INNER JOIN geolocation g ON g.EPA_srcID = q.EPA_srcID
                       WHERE q.rl_ppt NOT NULL
                       ORDER BY q.epa_srcid""")

        rows = cur.fetchall()
        cur.close()
        conn.close()
     
        df = pd.DataFrame(columns = ['samplingfeatureexternalidentifier', 'samplingfeaturecode',
                                     'samplingfeaturename', 'variablenamecv', 'variablecode',
                                     'cv_variable_term', 'datavalue', 'resultdatetime', 'datasetcode',
                                     'sitetypecv', 'propertyvalue', 'methodid', 'methodcode', 
                                     'network_class_id', 'network_subclass_id', 'resulttypecv',
                                     'sampledmediumcv', 'statuscv', 'unitid','unitabbreviation',
                                     'site_methodid', 'site_methodcode'])
                                     #'class', 'well_i_rat', 'intk_loc']) 
        for row in rows:
            variablecode = row[4]
            if variablecode == 'NVHOS':
                cv_variable_term = '801209-99-4'
            else:
                cv_variable_term = row[5]

            year,month,day = row[7].split('-')
            naive = datetime(int(year),int(month),int(day))
            eastern = pytz.timezone('America/New_York')
            aware = eastern.localize(naive, is_dst=None)
            resultdatetime = aware.replace(hour=12, minute=0, second=0)
            #is_dst(resultdatetime)
            
            samplingfeaturecode = row[1]+'_Specimen'
            datasetcode = 'T1Data_'+str(row[8])+'_'+samplingfeaturecode
            src_type = row[9]
            if src_type == 'G':
                sitetypecv = 'Well'
                methodid = 10004
                methodcode = 'mass-spec'
                site_methodid = 10003
                site_methodcode = 'grab-gw'
                network_class_id = 102
                network_subclass_id = 105
            elif src_type == 'S':
                sitetypecv = 'Stream'
                methodid = 10004
                methodcode = 'mass-spec'
                site_methodid = 10005
                site_methodcode = 'grab-sw'
                network_class_id = 102
                network_subclass_id = 106
            else:
                sitetypecv = 'None'
                
            resulttypecv = 'Measurement'
            sampledmediumcv = 'Liquid aqueous'
            statuscv = 'Public'
            unitid = 2793
            unitabbreviation = 'rl_ppt'
            
            df = df.append({'samplingfeatureexternalidentifier':row[0],'samplingfeaturecode':samplingfeaturecode,
                            'samplingfeaturename':row[2],'variablenamecv':row[3],'variablecode':variablecode,
                            'cv_variable_term':cv_variable_term,'datavalue':row[6],
                            'resultdatetime':resultdatetime,'datasetcode':datasetcode,'sitetypecv':sitetypecv, 
                            'propertyvalue':row[10],'methodid':methodid,'methodcode':methodcode,
                            'network_class_id':network_class_id,'network_subclass_id':network_subclass_id,
                            'resulttypecv':resulttypecv,'sampledmediumcv':sampledmediumcv,'statuscv':statuscv, 
                            'unitid':unitid,'unitabbreviation':unitabbreviation,'site_methodid':site_methodid,
                            'site_methodcode':site_methodcode},
                            #'class': row[11], 'well_i_rat': row[12], 'intk_loc': row[13]}, 
                           ignore_index = True)
        
        return df
    except sqlite3.Error as e:
        print(e)

In [10]:
df = getTeam1RlResultsMethods()
df.head()

,samplingfeatureexternalidentifier,samplingfeaturecode,samplingfeaturename,variablenamecv,variablecode,cv_variable_term,datavalue,resultdatetime,datasetcode,sitetypecv,...,methodcode,network_class_id,network_subclass_id,resulttypecv,sampledmediumcv,statuscv,unitid,unitabbreviation,site_methodid,site_methodcode
0,11606,NC0103010_Specimen,"SPARTA, TOWN OF",Perfluoro-2-ethoxypropanoic acid,PEPA,267239-61-2,1.0,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0103010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2793,rl_ppt,10003,grab-gw
1,11606,NC0103010_Specimen,"SPARTA, TOWN OF","1,1,2,2-tetrafluoro-2-(1,2,2,2-tetrafluoro-eth...",NVHOS,801209-99-4,5.0,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0103010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2793,rl_ppt,10003,grab-gw
2,11606,NC0103010_Specimen,"SPARTA, TOWN OF",N-(3-dimethylaminopropan-1-yl)perfluoro-1-hexa...,N-AP-FHxSA,50598-28-2,10.0,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0103010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2793,rl_ppt,10003,grab-gw
3,11606,NC0103010_Specimen,"SPARTA, TOWN OF",Perfluorononanesulfonic acid,PFNS,68259-12-1,2.0,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0103010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2793,rl_ppt,10003,grab-gw
4,11606,NC0103010_Specimen,"SPARTA, TOWN OF",Perfluorooctanoic acid,PFOA,335-67-1,2.0,2019-07-02 12:00:00-04:00,T1Data_Knappe_NC0103010_Specimen,Well,...,mass-spec,102,105,Measurement,Liquid aqueous,Public,2793,rl_ppt,10003,grab-gw


In [11]:
outfile = '../ncpfast-db/team1RlResultsMethods.csv'
df.to_csv(outfile, header=True, index=False)

In [13]:
len(df['datavalue'])

In [5]:
def loadTeam1ResultsMethods(units):
    
    if units.lower() == 'rl_ppt':
        infile = '../ncpfast-db/team1RlResultsMethods.csv'
    elif units.lower() == 'conc_ppt':
        infile = '../ncpfast-db/team1ConcResultsMethods.csv'
    else:
        sys.exit('Incorret units')

    df = pd.read_csv(infile)

    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        
        for index, row in df.iterrows():
            resultuuid = str(uuid4())
            samplingfeatureexternalidentifier = row[0]
            samplingfeaturecode = row[1]
            samplingfeaturename = row[2]
            variablenamecv = row[3]
            variablecode = row[4]
            cv_variable_term = row[5]
            datavalue = row[6]
            resultdatetime = row[7]
            datasetcode = row[8]
            sitetypecv = row[9]
            propertyvalue = row[10]
            methodid = row[11]
            methodcode = row[12]
            network_class_id = row[13]
            network_subclass_id = row[14]
            resulttypecv = row[15]
            sampledmediumcv = row[16]
            statuscv = row[17]
            unitid = row[18]
            unitabbreviation = row[19]
            site_methodid = row[20]
            site_methodcode = row[21]

            cur.execute("""INSERT INTO team1_results_methods(resultuuid,samplingfeatureexternalidentifier, 
                                                             samplingfeaturecode,samplingfeaturename,
                                                             variablenamecv,variablecode,cv_variable_term,
                                                             datavalue,resultdatetime,datasetcode,sitetypecv,
                                                             propertyvalue,methodid,methodcode,network_class_id,
                                                             network_subclass_id,resulttypecv,sampledmediumcv,
                                                             statuscv,unitid,unitabbreviation,site_methodid,
                                                             site_methodcode)
                            VALUES (%(vresultuuid)s,%(vsamplingfeatureexternalidentifier)s,%(vsamplingfeaturecode)s,
                                    %(vsamplingfeaturename)s,%(vvariablenamecv)s,%(vvariablecode)s,
                                    %(vcv_variable_term)s,%(vdatavalue)s,%(vresultdatetime)s,%(vdatasetcode)s,
                                    %(vsitetypecv)s,%(vpropertyvalue)s,%(vmethodid)s,%(vmethodcode)s,
                                    %(vnetwork_class_id)s,%(vnetwork_subclass_id)s,%(vresulttypecv)s,
                                    %(vsampledmediumcv)s,%(vstatuscv)s,%(vunitid)s,%(vunitabbreviation)s,
                                    %(vsite_methodid)s,%(vsite_methodcode)s)""",
                       {'vresultuuid':resultuuid,
                        'vsamplingfeatureexternalidentifier':samplingfeatureexternalidentifier,
                        'vsamplingfeaturecode':samplingfeaturecode,
                        'vsamplingfeaturename':samplingfeaturename,'vvariablenamecv':variablenamecv,
                        'vvariablecode':variablecode,'vcv_variable_term':cv_variable_term,'vdatavalue':datavalue,
                        'vresultdatetime':resultdatetime,'vdatasetcode':datasetcode,'vsitetypecv':sitetypecv,
                        'vpropertyvalue':propertyvalue,'vmethodid':methodid,'vmethodcode':methodcode,
                        'vnetwork_class_id':network_class_id,'vnetwork_subclass_id':network_subclass_id,
                        'vresulttypecv':resulttypecv,'vsampledmediumcv':sampledmediumcv,'vstatuscv':statuscv,
                        'vunitid': unitid,'vunitabbreviation': unitabbreviation, 'vsite_methodid': site_methodid,
                        'vsite_methodcode': site_methodcode})

        conn.commit()
        cur.execute("""ANALYZE team1_results_methods""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()      


In [6]:
loadTeam1ResultsMethods('conc_ppt')
loadTeam1ResultsMethods('rl_ppt')

In [2]:
def insertResultsMethods():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT samplingfeaturecode FROM team1_results_methods""")
        frows = cur.fetchall()
        
        for frow in frows:
            dsamplingfeaturecode = frow[0]

            cur.execute("""SELECT DISTINCT datasetcode, network_class_id, network_subclass_id, 
                                           resultdatetime, methodid, site_methodid
                           FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': dsamplingfeaturecode})
            try:
                drow = cur.fetchall()
                ddatasetcode = drow[0][0]
                network_class_id = drow[0][1]
                network_subclass_id = drow[0][2]
                resultdatetime = drow[0][3]
                methodid = drow[0][4]
                site_methodid = drow[0][5]
            except:
                sys.exit('Error on datasetcode')
                
            dlab = ddatasetcode.split('_')[1]
            
            if dlab == 'Knappe':
                try:
                    affiliationid = 10005
                    datasettitle = 'Team 1 stream and well measurements from Dr. Knappe Lab at NCSU at station '+
                    dsamplingfeaturecode[:-9]
                except:
                    sys.exit('Error on dsamplingfeaturecode')
            elif dlab == 'Ferguson':
                try:
                    affiliationid = 10001
                    datasettitle = 'Team 1 stream and well measurements from Dr. Ferguson Lab at Duke University at station '+
                    dsamplingfeaturecode[:-9]
                except:
                    sys.exit('Error on dsamplingfeaturecode')                    
            
            datasetuuid = str(uuid4())
            
            cur.execute("""INSERT INTO datasets (datasetuuid, datasetcode, datasettitle, datasetabstract,
                                                 dataseries, created_at, updated_at, network_class_id,
                                                 network_subclass_id)
                           VALUES (%(vdatasetuuid)s, %(vdatasetcode)s, %(vdatasettitle)s, 'TBD',
                                     6, '2021-01-20 11:30:00.000000-05', '2021-01-20 11:30:00.000000-05',
                                     %(vnetwork_class_id)s, %(vnetwork_subclass_id)s)
                                     RETURNING datasetid""",
                       {'vdatasetuuid': datasetuuid,'vdatasetcode': ddatasetcode, 'vdatasettitle': datasettitle,
                        'vnetwork_class_id': network_class_id, 'vnetwork_subclass_id': network_subclass_id})
            try:
                datasetid = cur.fetchone()[0]
            except:
                sys.exit('Error on datasetid')

            begin_datetime = resultdatetime.replace(hour=0, minute=0, second=0)
            end_datetime = resultdatetime.replace(hour=23, minute=59, second=59)
            validdatetime = resultdatetime.replace(hour=0, minute=0, second=0)
            endvaliddatetime = resultdatetime.replace(hour=23, minute=59, second=59)
            valuedatetime = resultdatetime

            cur.execute("""INSERT INTO actions(begin_datetime, end_datetime,methodid)
                           VALUES (%(vbegin_datetime)s, %(vend_datetime)s,%(vmethodid)s)
                           RETURNING actionid""",
                        {'vbegin_datetime': begin_datetime,'vend_datetime': end_datetime,'vmethodid': methodid})
            try:
                actionid = cur.fetchone()[0]
            except:
                sys.exit('Error on actionid')

            cur.execute("""INSERT INTO action_by(actionid, affiliationid)
                           VALUES (%(vactionid)s, %(vaffiliationid)s)""",
                        {'vactionid': actionid,'vaffiliationid': affiliationid})

            cur.execute("""SELECT samplingfeatureid FROM sampling_features 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': dsamplingfeaturecode})
            try:
                samplingfeatureid = cur.fetchone()[0]
            except:
                sys.exit('Error on samplingfeatureidid')
                
            cur.execute("""INSERT INTO feature_actions(actionid, samplingfeatureid)
                           VALUES (%(vactionid)s, %(vsamplingfeatureid)s)
                           RETURNING featureactionid""",
                        {'vactionid': actionid,'vsamplingfeatureid': samplingfeatureid})
            try:
                featureactionid = cur.fetchone()[0]
            except:
                sys.exit('Error on featureactionid')

            cur.execute("""INSERT INTO actions(begin_datetime, end_datetime,methodid)
                           VALUES (%(vbegin_datetime)s, %(vend_datetime)s,%(vsite_methodid)s)
                           RETURNING actionid""",
                        {'vbegin_datetime': begin_datetime,'vend_datetime': end_datetime,
                         'vsite_methodid': site_methodid})
            try:
                site_actionid = cur.fetchone()[0]
            except:
                sys.exit('Error on site_actionid')

            cur.execute("""SELECT to_samplingfeature_id FROM related_features 
                           WHERE from_samplingfeature_id = %(vsamplingfeatureid)s""",
                        {'vsamplingfeatureid': samplingfeatureid})
            try:
                site_samplingfeatureid = cur.fetchone()[0]
            except:
                sys.exit('Error on site_samplingfeatureidid')

            cur.execute("""INSERT INTO feature_actions(actionid, samplingfeatureid)
                           VALUES (%(vsite_actionid)s, %(vsite_samplingfeatureid)s)
                           RETURNING featureactionid""",
                        {'vsite_actionid': site_actionid,'vsite_samplingfeatureid': site_samplingfeatureid})
            try:
                site_featureactionid = cur.fetchone()[0]
            except:
                sys.exit('Error on site_featureactionid')

            cur.execute("""SELECT * FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': dsamplingfeaturecode})
            rows = cur.fetchall()
             
            for row in rows:
                resultidin = row[0]
                resultuuid = row[1]
                samplingfeatureexternalidentifier = row[2]
                samplingfeaturecode = row[3]
                samplingfeaturename = row[4]
                variablenamecv = row[5]
                variablecode = row[6]
                cv_variable_term = row[7]
                datavalue = row[8]
                #resultdatetime = row[9].strftime("%Y-%m-%d %H:%M:%S")
                resultdatetime = row[9]
                datasetcode = row[10]
                sitetypecv = row[11]
                propertyvalue = row[12]
                methodid = row[13]
                methodcode = row[14]
                vnetwork_class_id = row[15]
                vnetwork_subclass_id = row[16]
                resulttypecv = row[17]
                sampledmediumcv = row[18]
                statuscv = row[19]
                unitid = row[20]
                unitabbreviation = row[21]                
            
                cur.execute("""SELECT v.variableid FROM variables v
                               INNER JOIN cv_variable_names cvv
                               ON v.variablenamecv=cvv.name
                               WHERE cvv.term = %(vcv_variable_term)s""",
                            {'vcv_variable_term':cv_variable_term})
                try:
                    variableid = cur.fetchone()[0]
                except :
                    sys.exit('Error on variableid')


                cur.execute("""INSERT INTO results(resultuuid,resultdatetime,featureactionid,resulttypecv,
                                                   sampledmediumcv,statuscv,unitid,variableid,validdatetime,
                                                   endvaliddatetime)
                               VALUES (%(vresultuuid)s,%(vresultdatetime)s,%(vfeatureactionid)s,
                                       %(vresulttypecv)s,%(vsampledmediumcv)s,%(vstatuscv)s,%(vunitid)s,
                                       %(vvariableid)s,%(vvaliddatetime)s,%(vendvaliddatetime)s)
                               RETURNING resultid""",
                            {'vresultuuid':resultuuid,'vresultdatetime':resultdatetime,
                             'vfeatureactionid':featureactionid,'vresulttypecv':resulttypecv,
                             'vsampledmediumcv':sampledmediumcv,'vstatuscv':statuscv,'vunitid': unitid,
                             'vvariableid':variableid,'vvaliddatetime':validdatetime,
                             'vendvaliddatetime':endvaliddatetime})
                try:
                    resultid = cur.fetchone()[0]
                except:
                    sys.exit('Error on resultid')

                cur.execute("""INSERT INTO measurement_result_values(datavalue,valuedatetime,resultid)
                               VALUES (%(vdatavalue)s,%(vvaluedatetime)s,%(vresultid)s)""",
                            {'vdatavalue':datavalue,'vvaluedatetime':valuedatetime,'vresultid':resultid})

                cur.execute("""INSERT INTO dataset_results(resultid,datasetid)
                               VALUES (%(vresultid)s,%(vdatasetid)s)""",
                            {'vresultid':resultid,'vdatasetid':datasetid})
                
            conn.commit()

        cur.execute("""ANALYZE actions""")
        cur.execute("""ANALYZE feature_actions""")
        cur.execute("""ANALYZE measurement_result_values""")
        cur.execute("""ANALYZE results""")
        cur.execute("""ANALYZE dataset_results""")
        cur.execute("""ANALYZE datasets""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()  

In [3]:
insertResultsMethods()